# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.50,80,42,7.02,PN,1714363217
1,1,port-aux-francais,-49.3500,70.2167,4.19,97,100,20.18,TF,1714363218
2,2,iqaluit,63.7506,-68.5145,-20.50,94,89,1.49,CA,1714363219
3,3,mahibadhoo,3.7833,72.9667,28.34,77,100,5.91,MV,1714363220
4,4,ormara,25.2088,64.6357,29.11,29,1,8.05,PK,1714363220


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', size='Humidity', hover_cols=['City', 'Humidity'], 
                                       title='City Humidity', xlabel='Longitude', ylabel='Latitude')

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter cities based on ideal weather conditions
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 25) &
    (city_data_df["Humidity"] < 60) &
    (city_data_df["Cloudiness"] < 20) &
    (city_data_df["Wind Speed"] < 15) ]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,alice springs,-23.7000,133.8833,23.75,15,0,6.17,AU,1714363074
109,109,kutum,14.2000,24.6667,22.42,15,2,4.22,SD,1714363248
248,248,tanumah,27.1000,44.1333,21.30,50,1,2.53,SA,1714363286
261,261,aswan,24.0934,32.9070,23.76,22,0,4.97,EG,1714363289
272,272,waddan,29.1614,16.1390,24.42,29,18,4.14,LY,1714363291


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
34,alice springs,AU,-23.7000,133.8833,15,
109,kutum,SD,14.2000,24.6667,15,
248,tanumah,SA,27.1000,44.1333,50,
261,aswan,EG,24.0934,32.9070,22,
272,waddan,LY,29.1614,16.1390,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
print(geoapify_key)

154c32baae2f4a96adea6c3989acc03b


In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alice springs - nearest hotel: Aurora Alice Springs
kutum - nearest hotel: No hotel found
tanumah - nearest hotel: No hotel found
aswan - nearest hotel: Oscar Hotel
waddan - nearest hotel: No hotel found
paharpur - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
taunsa - nearest hotel: No hotel found
san ignacio rio muerto - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,alice springs,AU,-23.7000,133.8833,15,Aurora Alice Springs
109,kutum,SD,14.2000,24.6667,15,No hotel found
248,tanumah,SA,27.1000,44.1333,50,No hotel found
261,aswan,EG,24.0934,32.9070,22,Oscar Hotel
272,waddan,LY,29.1614,16.1390,29,No hotel found
377,paharpur,PK,32.1038,70.9724,51,No hotel found
409,crane,US,31.3974,-102.3501,19,No hotel found
438,taunsa,PK,30.7048,70.6505,49,No hotel found
448,san ignacio rio muerto,MX,27.4167,-110.2333,44,No hotel found
583,al jawf,SA,29.5000,38.7500,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    hover_cols=['City', 'Country', 'Hotel Name'],
    title='City Map with Information')


# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity,City,Country)